# Matching the chemicals found on the FEMA and JECFA websites with rdkit chemical representations

The [RDkit](http://www.rdkit.org/docs/Overview.html) is a chemical informatics toolkit. It allows for the calculation of chemical descriptors which can then be used as features for machine learning tasks. 

The scripts found here pairs each chemical of the merged FEMA and JECFA database created in [fema_jecfa_merge](fema_jecfa_merge.ipynb) with its RDkit representation. This will allow for the calculation of descriptor features for downstream machine learning tasks.

In [28]:
import os.path as path
import pickle

# Load merged FEMA-JECFA database
BASE_DATA_PATH = path.join(path.expanduser('~'),
                           'Dropbox',
                           'bymt',
                           'data_dumps',
                           'chem_project')

merged_chemicals_path = path.join(BASE_DATA_PATH, 'fema_jecfa_merge', 'merged_chemicals.pkl')
with open(merged_chemicals_path, 'rb') as f:
    merged_chemicals = pickle.load(f)

DATA_PATH = path.join(BASE_DATA_PATH, 'rdkit_chemical_matching')

RDKit does not allow for searches based on chemical nomenclature. Instead less ambiguous chemical representations such as [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) are used. To find the correct RDkit representation of the chemicals found in `merged_chemicals` these scripts use the chemspipy Python wrapper to interact with the [ChemSpider database](http://www.chemspider.com/). This database then allows for the calculation of the molecular weight and SMILES chemical representation of the molecules under consideration. With these two unambigous identifiers an RDkit molecule can be paired.

In [3]:
from chemspipy import ChemSpider
cs = ChemSpider('0201ba66-585d-4135-9e6b-d28ba4724fcf')
from rdkit import Chem
from rdkit.Chem import Descriptors
from inspect import getmembers, isfunction

In [ ]:
def same_chemical(results, mw):
    '''
    returns an rdkit chemical object if the chemicals in a chemspipy result list have:
    -the same molecular weight as the one given as 'mw' input
    -the same smiles representation as the chemspy results
    returns None otherwise
    '''
    if results.count == 0:
        return None, None

    smiles = []
    mws = []

    if results.count >= 1:
        for chemical in results:
            try:
                test1_mw = chemical.molecular_weight
                test1_mw = round(test1_mw)
                smiles_base = chemical.smiles
                chem_base = Chem.MolFromSmiles(smiles_base)
                test2_mw = Chem.Descriptors.MolWt(chem_base)
                test2_mw = round(test2_mw)
#                 print('Test1: {}, Test2: {}' .format(test1_mw, test2_mw))                
                if (mw == test1_mw and
                   test1_mw == test2_mw):
#                     print('Matched MWs')
                    return chem_base, chemical.csid                
                # If no mw is known determines if the results are internally consistent
                # If they are, it returns one of them
                if not mw:
                    smiles_temp = Chem.MolToSmiles(chem_base)
                    smiles.append(smiles_temp)
                    mw_temp = Chem.Descriptors.MolWt(chem_base)
                    mws.append(mw_temp)
                    if (len(set(smiles)) == 1 and
                    len(set(mws)) == 1):
#                         print('All results internally consistent, but no mw match')
                        return chem_base, chemical.csid           
            except:
                print(' MW EX', end=' ')
                continue
        else:
            return None, None    
    else:
        return None, None

In [52]:
def chem_search(dicto):
    '''
    returns an rdkit molecule and its chemspider id 
    after searching the chemspider database based on the items
    in the priority list.
    '''
    priority_list = [('fema', 'fema '),
                     ('jecfa', 'jecfa '), 
                     ('cas', ''),
                     ('name', ''),
                     ('chemical name', '')]
    
    for tup in priority_list:
        try:
            val = dicto.get(tup[0])
            val = str(val)
        except AttributeError:
            continue
            
        if val :
            search_string = tup[1] + val
#             print('searching for: {}' .format(search_string))
            results = cs.search(search_string)
            try:
                mw = dicto.get('molecular weight')
                mw = round(mw)
            except TypeError:
                mw = None                
#             print('stopped searching')
#             print('real MW: {}' .format(mw))
            rd, csid = same_chemical(results, mw)
            if rd:
                return rd, csid
            else:
                continue
    return None, None

In [29]:
from copy import deepcopy

def rdkit_printed_pairer(dicto_list):
    """
    Applies chem_search function to a list of individual chemical dictionaries.
    Displays a readout so that progress is known.
    """
    new_list = deepcopy(dicto_list)
    
    # Part of percentage display
    count = 0
    total = len(dicto_list)
    last_displayed = 0
    
    for dicto in new_list:
        rd, csid = chem_search(dicto)
        if rd:
            dicto['rdkit mol'] = rd
            dicto['csid'] = csid
        else:
            print(' {} failed' .format(dicto['name']), end=' ')
        
        # This noise is all about a nice display with percentage completed
        count += 1
        val = round((count / total) * 100)
        if (val % 5 == 0 and
            val != last_displayed):
            print('{:2.0f}%' .format(val), end = '.')
        else:
            print('.', end='')
        last_displayed = val
        
    return new_list

In [36]:
import pickle

def match_chunker(chunkable, filename='rdkit_chemicals.pkl', splits=10, chunk_list=None):
    """
    Splits the matching of individual chemical information into separate chunks.
    As each chunk is completed it is saved into an updated pickle file. 
    chunk_list can specify particular chunks to be processed. 
    """
    
    total = len(chunkable)
    
    # determine chunk size
    chunk_size, mod = total//(splits), total%splits
    if mod != 0:
        chunk_size = total//(splits-1)
        mod = total%chunk_size
        if (mod == 0 or
           mod < chunk_size/2): # This makes sure that the remainder is not too large
            chunk_size -= round(chunk_size/(2*splits))
    print('Chunk size: {}' .format(chunk_size))
    
    # Generate a list with the chunk indices so that if a specific chunk number is specified
    # it can be found and generated consistently
    start = 0
    end = chunk_size
    start_end_list = []
    while end != total:
        start_end_list.append((start, end))
        start += chunk_size
        end += chunk_size
        if end > total:
            end = total
    start_end_list.append((start,end))
    print('Number of chunks: {}' .format(len(start_end_list)))
    
    
    # This part does the actual matching
    rdkit_chemicals = []
    rdkit_chemicals_path = path.join(DATA_PATH, filename)
    
    if not chunk_list:
        iterable = enumerate(start_end_list)
    else:
        sub_is = [start_end_list[i] for i in chunk_list]
        iterable = enumerate(sub_is)
    
    for i, tup in iterable:
        print ('\nChunk number {}, start: {}, end: {}' .format(i, tup[0], tup[1]))
        chunk = chunkable[tup[0]:tup[1]]
        rdkit_chemicals += rdkit_printed_pairer(chunk)
        
        # Save after every chunk
        with open(rdkit_chemicals_path, 'wb') as f:
            pickle.dump(rdkit_chemicals, f, protocol=pickle.HIGHEST_PROTOCOL)
       
    return rdkit_chemicals

In [55]:
rdkit_chemicals = match_chunker(merged_chemicals)

Chunk size: 231
Number of chunks: 10

Chunk number 0, start: 0, end: 231
.......... 5%...........10%............15%.......... (-)-sclareol failed ..20%.. (2e,6e/z,8e)-n-(2-methylpropyl)-2,6,8-decatrienamide failed .........25%............30%...........35%............40%...........45%............50%...........55%............60%...........65%............70%............75%...........80%......... 1-phenyl-3 or 5-propylpyrazole failed ...85%...........90%.... 2,2,6,7-tetramethylbicyclo[4.3.0]nona-4,9(1)-dien-8-ol failed ........95%...........100%..
Chunk number 1, start: 231, end: 462
.......... 5%...........10%............15%............20%...........25%............30%...........35%............40%...........45%............50%...........55%............60%...........65%............70%............75%...........80%............85%...........90%............95%...........100%..
Chunk number 2, start: 462, end: 693
..... 2-methylbutyl 3-methylbutanoate failed ..... 5%...........10%............15%.

In [61]:
# Remove all the chemicals where an rdkit molecule was not found
trimmed_chemicals = []
for dicto in rdkit_chemicals:
    if 'rdkit mol' in dicto:
        trimmed_chemicals.append(deepcopy(dicto))
print('Chemicals with no rdkit molecule: {}' 
      .format(len(rdkit_chemicals)-len(trimmed_chemicals)))

Chemicals with no rdkit molecule: 23


In [63]:
trimmed_rdkit_chemicals_path = path.join(DATA_PATH, 'trimmed_rdkit_chemicals.pkl')

with open(trimmed_rdkit_chemicals_path, 'wb') as f:
    pickle.dump(trimmed_chemicals, f, protocol=pickle.HIGHEST_PROTOCOL)